In [ ]:
import os
import cv2
import gc
import matplotlib.pyplot    as plt
import numpy                as np
import matplotlib.patches   as mpatches
import plotly.graph_objects as go
import networkx             as nx
import cupy                 as cp
from scipy.stats       import skew, kurtosis, mode
from sklearn.cluster   import DBSCAN
from scipy.ndimage     import convolve, generic_filter
from cupyx.scipy.ndimage import convolve as cpconvolve
from cupyx.scipy.ndimage import binary_dilation
from scipy.signal import convolve2d
from collections import deque
from scipy.interpolate import interp1d

In [ ]:
#point_cloud_indices
name, heights, convolved_heights, intensity, x_to_i, i_to_x, y_to_i, i_to_y, sectioned = 0,1,2,3,4,5,6,7,8

window_height = 10

In [ ]:
class FileParser:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.box_folders = []

    def find_folders(self):
        for root, dirs, files in os.walk(self.root_dir):
                for dir_name in dirs:
                    if dir_name.startswith("Box9"):
                        #for i in range(5,16):
                        #adaptive_z_folder = os.path.join(root, dir_name, "AdaptiveZ_10mm")
                            adaptive_z_folder = os.path.join(root, dir_name, f"AdaptiveZ_10mm")

                            if os.path.isdir(adaptive_z_folder): 
                                for part_folder in os.listdir(adaptive_z_folder):
                                    full_part_path = os.path.join(adaptive_z_folder, part_folder)
                                    if os.path.isdir(full_part_path) and "Part" in part_folder:
                                        component_parameters_path = None
                                        lidar2xrf_path = None
                                        bpc_path = None
                                        real_dbg_path = None
                                        lidar_times_path = None
                                        for file_name in os.listdir(full_part_path):
                                            if file_name.endswith(".component_parameters.txt"):
                                                component_parameters_path = os.path.join(full_part_path, file_name)
                                            elif file_name.endswith(".lidar2xrf"):
                                                lidar2xrf_path = os.path.join(full_part_path, file_name)
                                            elif file_name.endswith(".bpc"):
                                                bpc_path = os.path.join(full_part_path, file_name)
                                            elif file_name.endswith("_intensity.png"):
                                                intensity_path = os.path.join(full_part_path, file_name)
                                            elif file_name.endswith("real.dbg"):
                                                real_dbg_path = os.path.join(full_part_path, file_name)
                                            elif file_name.endswith(".dbg"):
                                                lidar_times_path = os.path.join(full_part_path, file_name)

                                        self.box_folders.append((f"{part_folder} 10mm",component_parameters_path, lidar2xrf_path, bpc_path, intensity_path, real_dbg_path, lidar_times_path))

    def get_box_folders(self):
        self.find_folders()
        return self.box_folders
    
class BasicParser:
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.box_folders = []

    def find_folders(self):
        for root, dirs, files in os.walk(self.root_dir):
                for dir_name in dirs:
                    if dir_name.startswith("Core"):
                        folder = os.path.join(root,dir_name)
                      
                        if os.path.isdir(folder) and "0" in folder:
                            component_parameters_path = None
                            lidar2xrf_path = None
                            bpc_path = None
                            intensity_path = None
                            real_dbg_path = None
                            lidar_times_path = None
                            for file_name in os.listdir(folder):
                                if file_name.endswith(".component_parameters.txt"):
                                    component_parameters_path = os.path.join(folder, file_name)
                                elif file_name.endswith(".lidar2xrf"):
                                    lidar2xrf_path = os.path.join(folder, file_name)
                                elif file_name.endswith(".bpc"):
                                    bpc_path = os.path.join(folder, file_name)
                                elif file_name.endswith("_intensity.png"):
                                    intensity_path = os.path.join(folder, file_name)
                                elif file_name.endswith("real.dbg"):
                                    real_dbg_path = os.path.join(folder, file_name)
                                elif file_name.endswith(".dbg"):
                                    lidar_times_path = os.path.join(folder, file_name)

                            self.box_folders.append((folder,component_parameters_path, lidar2xrf_path, bpc_path, intensity_path, real_dbg_path, lidar_times_path))


    def get_box_folders(self):
        self.find_folders()
        return self.box_folders



parser = BasicParser(r"\\192.168.1.100\CoreScan3-2\Acquisitions\RnD\XRF 2.0\same core scans\AI21-TarcoreDoubleScans old xrf head\AI21-XRF1.0")
paths_list = parser.get_box_folders()

for name, component_parameters_path, lidar2xrf_path, bpc_path, intensity_path, real_dbg_path, lidar_times_path  in paths_list:
    print(f"Part: {name}")
    if component_parameters_path:
        print(f"  Component Parameters: {component_parameters_path}")
    if lidar2xrf_path:
        print(f"  LIDAR to XRF: {lidar2xrf_path}")
    if bpc_path:
        print(f"  BPC File: {bpc_path}")
    if intensity_path:
        print(f"  Intensity File: {intensity_path}")
    if lidar_times_path:
        print(f"  lidar time stamps file: {lidar_times_path}")
    if real_dbg_path:
        print(f"  real dbg file: {real_dbg_path}")


In [ ]:
def interpolate_x(table_file,lidar_file):
    with open(table_file) as file:
        lines = file.readlines()

        positions = []
        table_timestamps = []

        char = '-'

        for line in lines:
            if ("+") in line:
                char = '+'
            time = line.split('T')[1]
            x = float(line.split(',')[0])
            parts = time.split(':')
            t = float(parts[0]) * 3600 + float(parts[1]) * 60 + float(parts[2].split(char)[0]) 
            positions.append(x)
            table_timestamps.append(t)

    ordered_positions = []
    ordered_timestamps = []

    decreasing = (positions[-1] - positions[0]) < 0 

    if decreasing:
        for i in range(len(positions)-1):
            if positions[i + 1] < positions[i]:
                ordered_positions.append(positions[i])
                ordered_timestamps.append(table_timestamps[i])
    else:
         for i in range(len(positions)-1):
            if positions[i + 1] > positions[i]:
                ordered_positions.append(positions[i])
                ordered_timestamps.append(table_timestamps[i])


    with open(lidar_file) as file:
        lines = file.readlines()

        lidar_timestamps = []

        time = lines[0].split('T')[1]
        parts = time.split(':')
        min_t = float(parts[1]) * 60  + float(parts[0]) * 3600 + float(parts[2].split(char)[0]) 

        for line in lines:
            time = line.split('T')[1]
            parts = time.split(':')
            t = float(parts[0]) * 3600 + float(parts[1]) * 60 + float(parts[2].split(char)[0]) - min_t
            lidar_timestamps.append(t)


    lidar_timestamps = np.array(lidar_timestamps)
    ordered_timestamps = np.array(ordered_timestamps)[::-1]
    ordered_timestamps -= min_t
    ordered_positions = ordered_positions[::-1]

    ordered_timestamps = np.concatenate([ordered_timestamps[:2], ordered_timestamps[-2:]])
    ordered_positions = np.concatenate([ordered_positions[:2], ordered_positions[-2:]])

    interp_func = interp1d(ordered_timestamps, ordered_positions, kind="linear", fill_value=(ordered_positions[-1], ordered_positions[0]), bounds_error=False) 

    interpolated = interp_func(lidar_timestamps)
            
    return interpolated

def get_point_cloud(paths, y_window = window_height + 5, upsample_ratio = 2):
    print("loading",end = "... ")

    x_start = 50000
    x_stop = 0
    y_offset = 0
    with open(paths[1]) as file:
            lines = file.readlines()
            for line in lines:
                if "XRAY_DPP[Acquisition]#0.X.Start:" in line:
                    x_start = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Start:")[1].strip())
                if "XRAY_DPP[Acquisition]#0.X.Stop:" in line:
                    x_stop = (float)(line.split("XRAY_DPP[Acquisition]#0.X.Stop:")[1].strip())
                if "XRAY_DPP[Acquisition]#0.Y.Start:" in line:
                    y_offset = (float)(line.split("XRAY_DPP[Acquisition]#0.Y.Start:")[1].strip())
                    print(f"y_offset: {y_offset}")

    with open(paths[2]) as file:
        lines = file.readlines()
        transformation_matrix = np.array([list(map(float, line.strip().split(","))) for line in lines])

    imread = lambda fn: cv2.imread(fn, cv2.IMREAD_ANYDEPTH)
    
    point_cloud  = np.fromfile(paths[3], dtype=np.float32).reshape(-1, 3) 

    interpolated_x = interpolate_x(paths[5],paths[6])
    interpolated_x = np.repeat(interpolated_x, len(np.unique(point_cloud[:, 1])))

    intensity_map = imread(paths[4])
    
    print(f"{paths[0]} loaded, {point_cloud.shape[0]} points")
    print("trimming",end = "... ")
    
    intensity_values = np.reshape(intensity_map, (-1, 1))
    print(np.nanmax(intensity_values))
    intensity_cloud = np.hstack((point_cloud[:,:2], intensity_values))

    point_cloud = (np.hstack((point_cloud, np.ones((point_cloud.shape[0], 1)))) @ transformation_matrix.T)[:,:3]
    intensity_cloud = (np.hstack((intensity_cloud, np.ones((intensity_cloud.shape[0], 1)))) @ transformation_matrix.T)[:,:3]

   

    mask = (point_cloud[:,0] <= x_start) & (point_cloud[:,0] >= x_stop) & ((np.abs(point_cloud[:,1])) <= y_window)
    point_cloud = point_cloud[mask]
    intensity_cloud = intensity_cloud[mask]

    min_intensity = np.nanmax(intensity_cloud[:,2])
    min_z = np.nanmax(point_cloud[:,2])
    
    print(f"trimmed to {point_cloud.shape[0]} points")

    print("converting to arrays",end = "... ")

    minimum_x = point_cloud[np.argmin(np.abs(point_cloud[:,0] - x_stop)),0]

    point_cloud[:,0] -= minimum_x
    intensity_cloud[:,0] -= minimum_x
    
    x_values = np.unique(point_cloud[:,0])
    y_values = np.unique(point_cloud[:,1]) 

    x_range = len(x_values)

    if upsample_ratio > 1:
        index_step = (np.nanmedian(np.diff(x_values))) / upsample_ratio 
        index_steps = np.arange(int(round(np.nanmax(x_values)/index_step))+1) * index_step

        x_range = len(index_steps)
        x_value_dict = {x: np.argmin(np.abs(index_steps - x)) for x in x_values}
        known_indices = list(x_value_dict.values())
        known_x_values = np.array(list(x_value_dict.keys()), dtype=float)
        all_indices = np.arange(x_range)

        interp_func = interp1d(known_indices, known_x_values, kind="linear", fill_value="extrapolate")
        x_values = interp_func(all_indices)
    
    x_value_dict = {x: index for index,x in enumerate(x_values)}
    y_value_dict = {y: index for index,y in enumerate(y_values)}

    point_array = np.full((len(y_values), x_range),np.nan)
    intensity_array = np.full((len(y_values), x_range),np.nan)
    point_dictionary = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(point_cloud)}
    intensity_dictionary = {(row[0],row[1]): (index, row[2]) for index,row in enumerate(intensity_cloud)}

    for x in range (x_range):
        for y in range (len(y_values)):
            x_val = x_values[x]
            y_val = y_values[y]

            point_z = point_dictionary.get((x_val,y_val))
            intensity_z = intensity_dictionary.get((x_val,y_val))

            if point_z is not None:
                point_array[y,x] = point_z[1]
            else:
                point_array[y,x] = min_z
            if intensity_z is not None:
                intensity_array[y,x] = intensity_z[1]
            else:
                intensity_array[y,x] = min_intensity

    print(f"arrays built")
    if upsample_ratio > 1:
        print("upsampling",end = "... ")

        for y in range(len(y_values)):

            interpolated_z_values= []
            interpolated_i_values = []

            for dy in [-1,0,1]:
                y_dy = y + dy
                if (y_dy >= 0) & (y_dy < len(y_values)):
                    known_z = point_array[y_dy,known_indices]
                    known_i = intensity_array[y_dy,known_indices]
                    known_x = x_values[known_indices]
                    mask = ~np.isnan(known_z)
                    z_interp = interp1d(known_x[mask], known_z[mask], kind='linear', fill_value="extrapolate")
                    i_interp = interp1d(known_x[mask], known_i[mask], kind='linear', fill_value="extrapolate")

                    interpolated_z_values.append(z_interp(x_values))
                    interpolated_i_values.append(i_interp(x_values))

            point_array[y, :] = np.mean(interpolated_z_values, axis=0)
            intensity_array[y, :] = np.mean(interpolated_i_values, axis=0)
    
    kernel_y = np.array([[-1],[-2],[0],[2],[1]])
    convolved_array = np.abs(convolve(point_array,kernel_y))

    print(f"upsampled to {point_array.size} points")
    print(f"{paths[0]} finished \n")

    return [paths[0],point_array, convolved_array, intensity_array, x_value_dict, x_values, y_value_dict, y_values]


In [ ]:
#Get the point clouds
point_clouds = []

for paths in paths_list: 
    point_clouds.append(get_point_cloud(paths, y_window = window_height + 5 ,upsample_ratio=2))

In [ ]:
fig = plt.figure(figsize=(150, 7 * len(point_clouds)), dpi=150)  
gs = fig.add_gridspec(len(point_clouds) * 2, 1, hspace=0.6)  

for (i, point_cloud) in enumerate(point_clouds):
    name = point_cloud[0]
    cloud_1 = point_cloud[1]
    cloud_2 = point_cloud[3]
    

    ax = fig.add_subplot(gs[i * 2, 0])
    ax.imshow(np.flipud(cloud_1), cmap='jet', interpolation='nearest', alpha = 1)   
    ax.set_xlabel("X index")
    ax.set_ylabel("Y index")
    ax.set_title(f"Point Cloud: {name}", fontsize = 30 ) 

    ax = fig.add_subplot(gs[i* 2 + 1, 0])
    ax.imshow(np.flipud(cloud_2), cmap='binary', interpolation='nearest', alpha = 1)  
    ax.set_xlabel("X index")
    ax.set_ylabel("Y index") 


plt.show()


In [ ]:
import open3d as o3d
import numpy as np

pcds = []
point_cloud = point_clouds[0]

x = (list)(point_cloud[5]) * len(point_cloud[7])
y = np.repeat(point_cloud[7], len(point_cloud[5]))


array_z = point_cloud[1]
z = array_z.flatten()
array_i = point_cloud[3]
i = array_i.flatten()

pcd = o3d.geometry.PointCloud()

points = np.column_stack((x, y, -i))
pcd.points = o3d.utility.Vector3dVector(points)
#color = np.random.rand(3) 
#pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (len(points), 1)))
pcds.append(pcd)

o3d.visualization.draw_geometries(pcds)

x = np.diff(np.unique(x))